# Preparation

- install necessary packages
- setup the keys

In [1]:
!pip install -r requirements_congyu.txt

- Where to get openai_key: https://platform.openai.com/api-keys
- Where to get gemini_key: https://aistudio.google.com/app/apikey

In [41]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY', 'YourAPIKey')
google_api_key = os.getenv("GOOGLE_API_KEY", "YourAPIKey")

# print(openai_api_key)
# print(google_api_key)

or do these in terminal: 

```export OPENAI_API_KEY="value"```

```export GOOGLE_API_KEY="value"```

test API works or not

In [45]:
!(curl https://api.openai.com/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{"model": "gpt-3.5-turbo", "messages": [{"role": "user", "content": "Explain how AI works"}],"max_tokens": 100}')

{
  "id": "chatcmpl-ARddD9oiXrSwOufSzhWORCcnNQMU9",
  "object": "chat.completion",
  "created": 1731150563,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Artificial Intelligence, or AI, is a branch of computer science that aims to create machines or systems that can perform tasks that typically require human intelligence. AI works by using algorithms and data to mimic cognitive functions such as learning, problem-solving, perception, and decision-making.\n\nThere are different approaches to AI, including symbolic AI, which uses rules and logic to manipulate symbols, and machine learning, which relies on data and algorithms to improve performance over time. Machine learning is a subset of AI, and it",
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "length"
    }
  ],
  "usage": {
    "prompt_tokens": 12,
    "completion_tokens": 100,
    "total_tokens": 112,
   

In [47]:
!(curl -X POST "https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?key=$GEMINI_API_KEY"\
       -H 'Content-Type: application/json' \
       -d '{"contents":[{"parts":[{"text":"Explain how AI works"}]}]}')


{
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": "## How AI Works: A Simplified Explanation\n\nArtificial Intelligence (AI) is a broad field, but its core goal is to create machines that can **mimic human intelligence** in various tasks. While \"thinking\" like humans is a distant goal, AI has achieved remarkable progress in specific areas. Here's a simplified breakdown:\n\n**1. Learning from Data:**\n\n* AI systems are trained on **massive amounts of data**, like images, text, or sensor readings.\n* This data is used to **identify patterns and relationships**, enabling the AI to learn and make predictions.\n\n**2. Algorithms and Models:**\n\n* AI employs various **algorithms**, which are sets of instructions that guide the learning process.\n* These algorithms create **mathematical models** that represent the underlying patterns in the data.\n\n**3. Different Approaches:**\n\n* **Machine Learning (ML)** focuses on algorithms that can lear

# LangChain Cookbook 👨‍🍳👩‍🍳

# Schema

In [42]:
from langchain import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.schema import Document

In [43]:
prompt = PromptTemplate(input_variables=["text"], template="I am a new bee, please anwser me {text}")

text = prompt.format(text="how to use LangChain")
print(type(text), text)
print()


chat_msg = [
 SystemMessage(content="You are a nice AI bot that helps a user figure out where to travel in one short sentence"),
 HumanMessage(content="I like the beaches where should I go?"),
 AIMessage(content="You should go to Nice, France"),
 HumanMessage(content="What else should I do when I'm there?")
]
print(type(chat_msg[0]), chat_msg[0].content)
print()

document = Document(page_content="This is my document. It is full of text that I've gathered from other places",
         metadata={
             'my_document_id' : 234234,
             'my_document_source' : "The LangChain Papers",
             'my_document_create_time' : 1680013019
         })
print(type(document), document.page_content)
print()

<class 'str'> I am a new bee, please anwser me how to use LangChain

<class 'langchain_core.messages.system.SystemMessage'> You are a nice AI bot that helps a user figure out where to travel in one short sentence

<class 'langchain_core.documents.base.Document'> This is my document. It is full of text that I've gathered from other places



# Models

In [44]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

In [45]:
chat = ChatOpenAI(model='gpt-3.5-turbo', temperature=1, openai_api_key=openai_api_key)
# chat = ChatGoogleGenerativeAI(model='gemini-1.5-pro', temperature=0, google_api_key=google_api_key)

In [ ]:
ai_message = chat(chat_msg)
print(ai_message)
print()
print(ai_message.content)

# Chain

In [72]:
from langchain.chains import LLMChain, SimpleSequentialChain

In [63]:
template = """Your job is to come up with a classic dish from the area that the users suggests, and only respond the short dish name
% USER LOCATION
{user_location}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_location"], template=template)

# Holds my 'location' chain
location_chain = LLMChain(llm=chat, prompt=prompt_template)


In [64]:
location_chain.run("China")

'Peking Duck'

In [68]:
template = """Given a meal, give a short and simple recipe on how to make that dish at home. please respond in a very structured way.
% MEAL
{user_meal}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_meal"], template=template)

# Holds my 'meal' chain
meal_chain = LLMChain(llm=chat, prompt=prompt_template)

In [70]:
print(meal_chain.run("Peking Duck"))

1. Preheat the oven to 350°F.
2. Rinse and pat dry a whole duck. 
3. Rub the duck inside and out with a mixture of soy sauce, honey, hoisin sauce, and five-spice powder.
4. Place the duck on a roasting rack in a roasting pan.
5. Roast the duck in the oven for about 2 hours, turning and basting it every 30 minutes.
6. Increase the oven temperature to 425°F and roast for an additional 30 minutes to crisp up the skin.
7. Let the duck rest for 10 minutes before carving and serving with pancakes, scallions, cucumber, and hoisin sauce. Enjoy your homemade Peking Duck!


In [73]:
overall_chain = SimpleSequentialChain(chains=[location_chain, meal_chain], verbose=True)
print(overall_chain.run("Rome"))



> Entering new SimpleSequentialChain chain...
Cacio e Pepe
Ingredients:
- 8 oz spaghetti
- 1/2 cup grated Pecorino Romano cheese
- 1/2 cup grated Parmesan cheese
- 2 tsp freshly ground black pepper
- Salt to taste

Instructions:
1. Cook spaghetti in a large pot of salted boiling water until al dente. Reserve 1 cup of pasta water before draining.
2. In a separate pan, toast black pepper over low heat until fragrant.
3. Add 1/2 cup of pasta water to the pan and bring to a simmer.
4. Add cooked spaghetti to the pan and toss to coat in the pepper water.
5. Gradually add both cheeses, stirring constantly until melted and creamy. If needed, add more pasta water to achieve desired consistency.
6. Season with salt to taste.
7. Serve hot and enjoy your homemade Cacio e Pepe!

> Finished chain.
Ingredients:
- 8 oz spaghetti
- 1/2 cup grated Pecorino Romano cheese
- 1/2 cup grated Parmesan cheese
- 2 tsp freshly ground black pepper
- Salt to taste

Instructions:
1. Cook spaghetti in a large pot

In [98]:
def get_current_weather(location, unit):
    return 37.5

functions=[{
    "name": "get_current_weather",
    "description": "Get the current weather in a given location",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA"
            },
            "unit": {
                "type": "string",
                "enum": ["celsius", "fahrenheit"]
            }
        },
        "required": ["location"]
    }
}
]

output = chat(messages=[
    SystemMessage(content="You are an helpful AI bot"),
    HumanMessage(content="What’s the weather like in Boston right now?")
], functions=functions)

f_call_str = output.additional_kwargs["function_call"]
print(f_call_str)

f_call = "{func_name}(**{func_kwargs})".format(func_name=f_call_str["name"], func_kwargs=f_call_str["arguments"])
print(f_call)
eval(f_call)

{'arguments': '{"location":"Boston","unit":"celsius"}', 'name': 'get_current_weather'}
get_current_weather(**{"location":"Boston","unit":"celsius"})


37.5

In [82]:
def get_current_weather(location, unit):
    return 37.5

output = chat(messages=
     [
         SystemMessage(content="You are an helpful AI bot"),
         HumanMessage(content="What’s the weather like in Boston right now?")
     ],
     functions=[{
         "name": "get_current_weather",
         "description": "Get the current weather in a given location",
         "parameters": {
             "type": "object",
             "properties": {
                 "location": {
                     "type": "string",
                     "description": "The city and state, e.g. San Francisco, CA"
                 },
                 "unit": {
                     "type": "string",
                     "enum": ["celsius", "fahrenheit"]
                 }
             },
             "required": ["location"]
         }
     }
     ]
)

f_call_str = output.additional_kwargs["function_call"]
print(f_call_str)

f_call = "{func_name}(**{func_kwargs})".format(func_name=f_call_str["name"], func_kwargs=f_call_str["arguments"])
print(f_call)
eval(f_call)

{'arguments': '{"location":"Boston","unit":"celsius"}', 'name': 'get_current_weather'}
get_current_weather(**{"location":"Boston","unit":"celsius"})


37.5